# In this notebook, we will load our cleaned data set (train) and join with dimension tables for plots 

Take our cleaned train and use our additional dimension tables (items, stores, transactions, holiday_events, oil) to merge for visualizations